In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
import re 

#initialize_webdriver
def start_driver():
    driver = webdriver.Chrome()
    driver.get('https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31')
    time.sleep(5)
    return driver

#scrape_movie_data
def scrape_genre_data(genre):
    driver = start_driver()
    scroll_attempts = 0
    while scroll_attempts < 5:
        driver.execute_script("window.scrollBy(0, 100);")
        time.sleep(1)

        try:
            genre_section = driver.find_element(By.XPATH, "//div[contains(text(), 'Genre')]")
            genre_section.click()
            time.sleep(2)
            break  #stop_scrolling_button_found_means
        except:
            scroll_attempts += 1
            continue

    #scroll_and_find_genre_button
    scroll_attempts = 0
    while scroll_attempts < 3:
        driver.execute_script("window.scrollBy(0, 100);")
        time.sleep(1.5)
        try:
            print(f"Selecting genre: {genre}")
            genre_button = driver.find_element(By.XPATH, f"//button[span[contains(text(), '{genre}')]]")
            genre_button.click()
            time.sleep(2)
            break
        except:
            print(f"{genre} genre not found")
            scroll_attempts += 1
            continue

    movies = []
    while len(movies) < 200: #this_will_stop_count_meet
        new_movies = 0
        while new_movies < 50 and len(movies) < 200: #this_will_stop_every_50_counts
            movie_elements = driver.find_elements(By.XPATH, "//a[h3[@class='ipc-title__text']]")

            for movie in movie_elements:
                movie_name = movie.text.strip()
                movie_name = re.sub(r'^\d+\.\s*', '', movie_name) 
                if movie_name and movie_name not in [m['Movie Name'] for m in movies]: #check_not_already_in_list
                    try:
                        raw_duration = movie.find_element(By.XPATH, "(./ancestor::li//span[contains(@class, 'dli-title-metadata-item')])[2]").text.strip()
                        movie_duration = convert_duration(raw_duration) 
                    except:
                        movie_duration = "N/A"

                    try:
                        movie_rate = movie.find_element(By.XPATH, "./ancestor::li//span[contains(@class, 'ipc-rating-star--rating')]").text.strip()
                        movie_rating = float(movie_rate)
                    except:
                        movie_rating = "N/A"

                    try:
                        movie_votes = movie.find_element(By.XPATH, "./ancestor::li//span[contains(@class, 'ipc-rating-star--voteCount')]").text.strip()
                        movie_votes = re.sub(r"[()]", "", movie_votes)
                        if 'K' in movie_votes:
                            movie_votes = int(float(movie_votes.replace('K', '')) * 1000)
                        elif 'M' in movie_votes:
                            movie_votes = int(float(movie_votes.replace('M', '')) * 1000000)
                        else:
                            movie_votes = int(movie_votes)
                    except:
                        movie_votes = "N/A"

                    movies.append({
                        "Genre": genre,
                        "Movie Name": movie_name,
                        "Duration": raw_duration,
                        "Duration_Minutes": movie_duration,
                        "Rating": movie_rating,
                        "Votes": movie_votes
                    })
                    new_movies += 1
                    print({movie_name,raw_duration,movie_duration,movie_rating,movie_votes})

                    #scroll_each_movie_collected
                    driver.execute_script("window.scrollBy(0, 200);")
                    time.sleep(1)


        #scroll_for_load_more_button
        scroll_up_attempts = 0
        while scroll_up_attempts < 5 and len(movies) < 200:
            driver.execute_script("window.scrollBy(0, -100);")
            time.sleep(1)
            try:
                load_more_button = driver.find_element(By.XPATH, "//*[@id='__next']/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button")
                load_more_button.click()
                time.sleep(2)
                break
            except:
                scroll_up_attempts += 1
                continue

    #scroll_for_back_to_top_button
    scroll_up_attempts = 0
    while len(movies) >= 200 and scroll_up_attempts < 5:
        try:
            back_to_top = driver.find_element(By.XPATH, "//*[@id='__next']/button")
            back_to_top.click()
            time.sleep(3)
            break
        except:
            scroll_up_attempts += 1
            continue

    driver.quit()

    df = pd.DataFrame(movies)
    df.to_csv(f"{genre.lower()}_moviesss.csv", index=False)
    print(f"Data saved as {genre}_movies.csv")


selected_genre = "Action"  #change_genre_name_here_to_collect_next_genre_data
scrape_genre_data(selected_genre)

#convert_duration_function
def convert_duration(duration_text):
    hours = 0
    minutes = 0
    match = re.match(r"(?:(\d+)h)?\s*(?:(\d+)m)?", duration_text)
    if match:
        if match.group(1):
            hours = int(match.group(1)) * 60
        if match.group(2):
            minutes = int(match.group(2))
    return hours + minutes